In [1]:
# Konfigurasi Spark
import os
import sys

# 1. Mengeset variabel yang menyimpan lokasi di mana Spark diinstal
# spark_path = "D:/spark-2.1.0-bin-hadoop2.7"
spark_path = "D:/its/big_data/spark/"

# 2. Menentukan environment variable SPARK_HOME
os.environ['SPARK_HOME'] = spark_path

# 3. Simpan lokasi winutils.exe sebagai environment variable HADOOP_HOME
os.environ['HADOOP_HOME'] = spark_path

# 4. Lokasi Python yang dijalankan --> punya Anaconda
#    Apabila Python yang diinstall hanya Anaconda, maka tidak perlu menjalankan baris ini.
os.environ['PYSPARK_PYTHON'] = sys.executable

# 5. Konfigurasi path library PySpark
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.4-src.zip")

# 6. Import library Spark
#    Dua library yang WAJIB di-import adalah **SparkContext** dan **SparkConf**.
from pyspark import SparkContext
from pyspark import SparkConf
import numpy as np

# Setting konfigurasi (opsional)
conf = SparkConf()
conf.set("spark.executor.memory", "2g")
conf.set("spark.cores.max", "4")

sc = SparkContext("local", conf=conf)
#    Apabila berhasil, maka ketika sc di-print akan mengeluarkan nilai <pyspark.context.SparkContext object>
print sc

In [2]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

# Load and parse the data
data = sc.textFile("test.data")
ratings = data.map(lambda l: l.split(','))\
    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

In [3]:
ratings.take(5)

[Rating(user=1, product=1, rating=5.0),
 Rating(user=1, product=2, rating=1.0),
 Rating(user=1, product=3, rating=5.0),
 Rating(user=1, product=4, rating=1.0),
 Rating(user=2, product=1, rating=5.0)]

In [15]:
# Build the recommendation model using Alternating Least Squares
rank = 10
numIterations = 10
model = ALS.train(ratings, rank, numIterations)

In [24]:
# Evaluate the model on training data
testdata = ratings.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
predictions.collect()

[((4, 4), 4.996461434017894),
 ((4, 1), 1.0001724611154135),
 ((4, 3), 1.0001724611154135),
 ((4, 2), 4.996461434017894),
 ((1, 4), 1.0002706804879038),
 ((1, 1), 4.9959701202607425),
 ((1, 3), 4.9959701202607425),
 ((1, 2), 1.0002706804879038),
 ((3, 4), 4.996461434017894),
 ((3, 1), 1.0001724611154135),
 ((3, 3), 1.0001724611154135),
 ((3, 2), 4.996461434017894),
 ((2, 4), 1.0002706804879038),
 ((2, 1), 4.9959701202607425),
 ((2, 3), 4.9959701202607425),
 ((2, 2), 1.0002706804879038)]

In [25]:
ratesAndPreds.collect()

[((1, 3), (5.0, 4.9959701202607425)),
 ((3, 3), (1.0, 1.0001724611154135)),
 ((3, 1), (1.0, 1.0001724611154135)),
 ((4, 4), (5.0, 4.996461434017894)),
 ((1, 1), (5.0, 4.9959701202607425)),
 ((2, 2), (1.0, 1.0002706804879038)),
 ((4, 2), (5.0, 4.996461434017894)),
 ((2, 4), (1.0, 1.0002706804879038)),
 ((1, 2), (1.0, 1.0002706804879038)),
 ((3, 2), (5.0, 4.996461434017894)),
 ((3, 4), (5.0, 4.996461434017894)),
 ((1, 4), (1.0, 1.0002706804879038)),
 ((2, 3), (5.0, 4.9959701202607425)),
 ((2, 1), (5.0, 4.9959701202607425)),
 ((4, 3), (1.0, 1.0001724611154135)),
 ((4, 1), (1.0, 1.0001724611154135))]

In [26]:
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 7.21609767136e-06


In [28]:
# Save and load model
model.save(sc, "myCollaborativeFilter")
sameModel = MatrixFactorizationModel.load(sc, "myCollaborativeFilter")